In [ ]:
from mnist import MNIST
import numpy as np
import sklearn.metrics as metrics

"""
Change this code however you want.
"""
NUM_CLASSES = 10
NUM_HIDDEN = 200
INPUT_DIM = 784

def load_dataset():
    mndata = MNIST('./data/')
    X_train, labels_train = map(np.array, mndata.load_training())
    # The test labels are meaningless,
    # since you're replacing the official MNIST test set with our own test set
    X_test, _ = map(np.array, mndata.load_testing())
    # Remember to center and normalize the data...
    return X_train, labels_train, X_test

def sigmoid(x):
    from scipy.special import expit
    "Numerically-stable sigmoid function."
    return expit(x)#1 / (1 + np.exp(-x))

def dsigmoid(x):
    return sigmoid(x)*(1 - sigmoid(x))

def softmax(x):
    """Compute the softmax of vector x in a numerically stable way."""
    shiftx = x - np.max(x)
    exps = np.exp(shiftx)
    return exps / np.sum(exps)

def relu(arr):
	return np.maximum(arr, 0)

def drelu(arr):
    arr[arr>0] = 1
    arr[arr<0] = 0
    return arr

def one_hot(labels_train):
    '''Convert categorical labels 0,1,2,....9 to standard basis vectors in R^{10} '''
    return np.eye(NUM_CLASSES)[labels_train]

def preprocess(X_train):
    from sklearn import preprocessing
    return preprocessing.scale(X_train)

def predict(W, V, X):
    ''' From model and data points, output prediction vectors '''
    #forward pass: input to hidden layer
    s_h = X @ V.T # N x d+1 x d+1 x h = N x h
    h = relu(s_h)
    h_bias = np.concatenate((h, np.ones((h.shape[0],1))), axis = 1) #1xh+1

    #forward pass: hidden to output layer
    s_z = h_bias @ W.T # N x h x h+1 x k = N x k
    z = softmax(s_z) # N x k
    return np.argmax(z, axis=1)

def standardize(X):
    return (X - np.mean(X)) / np.std(X)

In [ ]:
"""Deskewing, taken from piazza, https://fsix.github.io/mnist/Deskewing.html"""
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import math
from scipy.ndimage import interpolation

def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix
def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    return interpolation.affine_transform(image,affine,offset=offset)

from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    assert len(image.shape)==2

    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing='ij')
    indices = np.reshape(x+dx, (-1, 1)), np.reshape(y+dy, (-1, 1))
    
    return map_coordinates(image, indices, order=1).reshape(shape)
def deskewAll(X):
    currents = []
    for i in range(len(X)):
        currents.append(deskew(X[i].reshape(28,28)).flatten())
    return np.array(currents)

In [ ]:
"""Loading the dataset"""
X_train, labels_train, X_test = load_dataset()
X_train_original, labels_train_original, X_test_original = X_train, labels_train, X_test

In [ ]:
"""Deskewing the training and testing sets"""
X_train, X_test = deskewAll(X_train), deskewAll(X_test)

In [ ]:
"""Randomly selecting 50000 training points and 10000 validation points"""
train_idx = np.random.choice(X_train.shape[0], X_train.shape[0]-10000, replace=False)
val_idx = np.delete(np.arange(X_train.shape[0]), train_idx)
labels_train_cut = labels_train[train_idx]
labels_val = labels_train[val_idx]
y_train = one_hot(labels_train)
X_train = X_train.astype(float)
X_train_new = preprocess(X_train)
X_train_added = np.concatenate((X_train_new, np.ones((X_train_new.shape[0],1))), axis = 1)
X_train_cut = X_train_added[train_idx]
X_val = X_train_added[val_idx]
y_train_cut = y_train[train_idx]
y_val = y_train[val_idx]

In [ ]:
epsilon = 0.01 #standard deviation for initial normal distribution
plotFreq = 1000 #plot every plotFreq computations
def train_sgd(X_train, y_train, y_labels, alpha=0.1, reg=0, 
              num_iter=15, momentum=False, decay=-10e-6, graph=False, V=None, W=None):
    ''' Build a model from X_train -> y_train using stochastic gradient descent '''
    if V is None or W is None: #Allows for checkpointing
        V = np.random.normal(scale = epsilon, size = (NUM_HIDDEN, INPUT_DIM + 1))
        W = np.random.normal(scale = epsilon, size = (NUM_CLASSES, NUM_HIDDEN + 1))
    if momentum:
        d_W_old = W
        d_V_old = V
    loss_lst = [] #accumulate losses for analysis
    accuracy_lst = [] #accumulate training accuracy for analysis
    count = 0
    for num in range(num_iter): 
        #num_iter is the number of times we pass through the ENTIRE datasets
        order = np.arange(X_train.shape[0]) 
        np.random.shuffle(order) #go through dataset in random order
        for i in order:
            X_i = X_train[i].reshape((1,785)) #current datapoint
            y_i = y_train[i].reshape((1,10)) #current labels
            
            #forward pass: input to hidden layer
            s_h = X_i.dot(V.T) # 1 x d+1 x d+1 x h = 1 x h
            #h = sigmoid(s_h)
            h = relu(s_h) #1 x h
            h_bias = np.concatenate((h, np.ones((h.shape[0],1))), axis = 1) #1xh+1
            #forward pass: hidden to output layer
            s_z = h_bias.dot(W.T) # 1 x h+1 x h+1 x k
            z = softmax(s_z) #1 x k
            
            if graph:
                if count % plotFreq == 0:
                    #compute loss
                    J = -np.sum(y_i*np.log(z))
                    loss_lst.append(J)
                    #compute accuracy
                    pred_labels_train = predict(W, V, X_train)
                    accuracy = metrics.accuracy_score(y_labels, pred_labels_train)
                    print("accuracy:", accuracy, "loss", J)
                    accuracy_lst.append(accuracy)
                    #reduce learning rate
                    alpha = 0.99*alpha

            #backprop: output to hidden layer
            d_z = z - y_i # 1 x 10
            d_W = d_z.T.dot(h_bias) # 10 x 1 x 1 x h+1 = 10 x h+1

            #backprop: hidden to input layer
            W_dropped = W[:,:-1] #drop the bias that is only relevant to the previous step
            #1 x 10 @ 10 x 200 * 1 x 200 .T @ 1 x 785 = 200 x 785
            d_V = (d_z.dot(W_dropped) * drelu(s_h)).T.dot(X_i) 
            
            if momentum:
                d_W = momentum*d_W_old+(1-momentum)*d_W
                d_V = momentum*d_V_old+(1-momentum)*d_V
                d_W_old = d_W
                d_V_old = d_V
                
            #update weights
            W -= alpha*d_W
            V -= alpha*d_V
            count += 1
    return W, V, loss_lst, accuracy_lst

In [ ]:
for a in [2e-3]:
    for dec in [0.99]:
        #1e-4, 2e-4, 3e-4, 4e-4, 5e-4, 6e-4, 7e-4, 8e-4, 9e-4, 1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3
        W, V, loss_lst, accuracy_lst = train_sgd(X_train_cut, y_train_cut, labels_train_cut, alpha=a, num_iter=3, momentum=False, decay=dec, graph=True)
        pred_labels_train = predict(W, V, X_train_cut)
        pred_labels_val = predict(W, V, X_val)
        print("Stochastic gradient descent w/ alpha=", a, "decay=", dec)
        print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train_cut, pred_labels_train)))
        print("Val accuracy: {0}".format(metrics.accuracy_score(labels_val, pred_labels_val)))

        import matplotlib.pyplot as plt  
        %matplotlib inline
        loss_arr = np.array(loss_lst)
        accuracy_arr = np.array(accuracy_lst)
        x = np.arange(len(loss_arr))*plotFreq
        plt.subplot(2, 1, 1)
        plt.plot(x, loss_arr, 'y.-')
        plt.xlabel('#Iterations')
        plt.ylabel('Training Loss')
        plt.subplot(2, 1, 2)
        plt.plot(x, accuracy_arr, 'r.-')
        plt.xlabel('#Iterations')
        plt.ylabel('Prediction Accuracy')
        plt.savefig("plots.png")

In [ ]:
X_test = X_test.astype(float)
X_test_new = preprocess(X_test)
X_test_added = np.concatenate((X_test_new, np.ones((X_test_new.shape[0],1))), axis = 1)

In [ ]:
"""Ensemble Learning, Training 5 Identical Neural Networks"""
all_predicts = []
all_W = []
all_V = []
for i in range(5):
    #1e-4, 2e-4, 3e-4, 4e-4, 5e-4, 6e-4, 7e-4, 8e-4, 9e-4, 1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3
    W, V, loss_lst, accuracy_lst = train_sgd(X_train_added, y_train, labels_train, alpha=2e-3, num_iter=10, momentum=False, decay=0.99, graph=False)
    pred_labels_train = predict(W, V, X_train_added)
    pred_labels_test = predict(W, V, X_test_added)
    print("Stochastic gradient descent w/ alpha=", a)
    print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train, pred_labels_train)))
    all_predicts.append(pred_labels_test)
    all_W.append(W)
    all_V.append(V)

In [ ]:
def ensemble(all_predicts):
    #all_predicts is a list of predictions
    #all_predicts is N x 10000
    from scipy import stats
    return stats.mode(all_predicts, axis=0)[0]

In [ ]:
ensemble_predicts = ensemble(all_predicts)

In [ ]:
ensemble_predict_list = ensemble_predicts.tolist()[0]

In [ ]:
"""CONTEST SUBMISSION"""
import csv

f = open("ashton_teng_predictions.csv", 'wt')
try:
    writer = csv.writer(f)
    writer.writerow(('Id', 'Category'))
    for i in range(len(pred_labels_test)):
        writer.writerow( (i+1, ensemble_predict_list[i]) )
finally:
    f.close()